In [1]:
## Import libraries
import numpy as np
import pandas as pd
import yaml
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

In [2]:
def remove_na_feature(df,threshold=50):
    """
    Function to generate report of features with NaN and their corresponding %
    and remove features with null value % greater than or equal to a threshold
    
    # Arguments
        df: Input dataframe
        threshold: Threshold value % to remove columns with Nan. Default is 50%
        
    # Returns
        dataframe: Dataframe with NaN columns removed
    """
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    report = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    report = report[report.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1).reset_index()
    report.columns=['Feature','Missing Values','% of Total Values']
    print(f'Data has {df.shape[1]} columns.\n'
          f'There are {report.shape[0]} columns that have missing values.')
    report.to_csv('Null_values_report.csv',index=False)
    print('Null value report saved as Null_values_report.csv!! \n')
    
    feature_nan= report.loc[report['% of Total Values']>=threshold,:]['Feature']
    print(f'{len(feature_nan)} features have nulls greater than the threshold and will be dropped.')
    dropped= pd.DataFrame(feature_nan.values,columns=['Dropped_features']) 
    dropped.to_csv('Null_dropped_report.csv',index=False)
    print('Dropped features saved in report Null_Dropped_Report.csv!! \n')
    df.drop(columns=feature_nan,inplace=True)
    
    return df

def impute_na(df,impute_method='median'):
    """
    Function to impute missing values with mean, first seen, last seen 
    or avg of first seen and last seen of column values
    
    # Arguments
        df: Input dataframe
        impute_method: Method to impute NaN in columns. Default is 'median'
        
    # Returns
        dataframe: Dataframe with NaN in columns imputed
    """
    if impute_method == 'median':
        df.fillna(df.median(), inplace=True) 
    
    elif impute_method == 'mean':
        df.fillna(df.mean(), inplace=True) 
    
    elif impute_method=='fill_forward':
        df.fillna(method='ffill',inplace=True)
    
    elif impute_method=='fill_backward':
        df.fillna(method='bfill',inplace=True)

    elif impute_method=='fill_fb_mean':
        df=(df.ffill()+df.bfill())/2
    
    print(f'Imputation performed using {impute_method}\n')
        
    return df.bfill().ffill()

In [3]:
def outlier_count_standard_deviation(df,num_of_std=3):
    """
    Function to treat outlier
    
    # Arguments
        df: Input dataframe
        num_of_std: Number of standard deviations
        
    # Returns
        dataframe: dataframe with outliers treated
    """
    print('Outliers are capped between [upper-lower] bound using standard deviation method.\n')
    
    for col in df.columns:
        if df.loc[:,[col]].dtypes.values != 'object':
            data = df[col]
            data_mean, data_std = np.mean(data), np.std(data)
            # identify outliers
            cut_off = data_std * num_of_std
            lower, upper = data_mean - cut_off, data_mean + cut_off
            # Impute Outliers
            df[col] = np.where((df[col]<lower),lower,np.where((df[col]>upper),upper,df[col]))
            
    return df

In [4]:
def remove_low_variance(df,threshold=0):
    """
    Function to remove features with low variance
    
    # Arguments
        df: Input dataframe
        threshold: Low variance threshold
        
    # Returns
        dataframe: Dataframe with low variance columns removed
    """
    normalized_df=(df-df.mean())/df.std()
    normalized_df=normalized_df.fillna(0)
    constant_filter = VarianceThreshold(threshold=threshold)  
    constant_filter.fit(normalized_df)
    columns_with_variance= normalized_df.columns[constant_filter.get_support()]
    constant_columns = [column for column in normalized_df.columns  
                        if column not in columns_with_variance]
    report= pd.DataFrame(constant_columns,columns=['Low_Variance_Feature']) 
    report.to_csv('Low_Variance_Features_Report.csv',index=False)
    print(f'{len(constant_columns)} features with {threshold} variance found and dropped')
    print('Dropped Low variance features saved in Low_Variance_Features_Report.csv!!\n')
    df = df.drop(labels=constant_columns, axis=1)
    
    return df

In [5]:
def sanity_checks(df,null_threshold,impute_method,num_of_std,variance_threshold):
    """
    Function to perform preliminary data cleaning and sanity checks
    
    # Arguments
        df: Input dataframe
        null_threshold: Threshold value % to remove columns with NaN
        impute_method: Method to impute NaN in data
        num_of_std: Number of standard deviations
        variance_threshold:Low variance threshold
        
    # Returns
        dataframe: data with sanity checks treated
    """
    df=remove_na_feature(df,null_threshold)
    df=impute_na(df,impute_method)
    df=outlier_count_standard_deviation(df,num_of_std)
    final_df=remove_low_variance(df,variance_threshold)
    return final_df

In [6]:
def train_valid_test_split(df,train_size=0.9,val_size=0.05):
    """
    Function to split data into train, validation and test sets
    
    # Arguments
        df: Input dataframe
        train_size: Percentage of rows to keep in train. Default is 0.9
        val_size: Percentage of rows to keep in validation. Default is 0.05
        
    # Returns
        3 dataframe: Returns the training, validation and test datasets
    """
    train, validate, test = np.split(df, [int(train_size*len(df)), int((train_size+val_size)*len(df))])
    return train, validate, test

In [7]:
#Read the parameters file  
with open("params.yml", 'r') as f:
    params = yaml.safe_load(f)

#Path to dataset    
data_path= params['input_data_path']

#Sanity check parameters
sanity_check_dict = params['sanity_check']
null_threshold = sanity_check_dict['null_threshold']
num_of_std = sanity_check_dict['num_of_std']
variance_threshold = sanity_check_dict['variance_threshold']
impute_method = sanity_check_dict['impute_method']

#Train, Validation and Test split parameters
split_dict = params['train_valid_test_split']
train_size = split_dict['train_size']
val_size = split_dict['val_size']

In [16]:
#Read dataset
df = pd.read_csv(data_path)

In [9]:
%%time
#Perform sanity checks
final_df=sanity_checks(df,null_threshold,impute_method,num_of_std,variance_threshold)

Data has 4298 columns.
There are 4298 columns that have missing values.
Null value report saved as Null_values_report.csv!! 

1252 features have nulls greater than the threshold and will be dropped.
Dropped features saved in report Null_Dropped_Report.csv!! 

Imputation performed using fill_forward

Outliers are capped between [upper-lower] bound using standard deviation method.

0 features with 0 variance found and dropped
Dropped Low variance features saved in Low_Variance_Features_Report.csv!!

CPU times: user 8.06 s, sys: 32.5 ms, total: 8.09 s
Wall time: 8.12 s


In [10]:
#final_df: data after performing sanity checks
final_df.head(5)

year  month  High000075.KS  Low000075.KS  Open000075.KS  Close000075.KS  \
0  2007.0    1.0    29095.03065   28337.10757    28634.98438     29012.06926   
1  2007.0    2.0    27798.58185   27415.33612    27696.08185     27669.34553   
2  2007.0    3.0    26590.06885   26035.55029    26406.25635     26344.97522   
3  2007.0    4.0    27432.12305   27070.13867    27149.32332     27398.19231   
4  2007.0    5.0    29087.31417   28503.74284    28657.80943     28958.93331   

   High000157.KS  Low000157.KS  Open000157.KS  Close000157.KS  ...  \
0    22272.72727   21654.54545    22004.54545     22004.54545  ...   
1    21921.05263   21218.42105    21547.36842     21584.21053  ...   
2    27935.00000   26882.50000    27230.00000     27770.00000  ...   
3    36790.47619   35959.52381    36276.19048     36509.52381  ...   
4    47566.66667   46016.66667    46473.80952     47185.71429  ...   

   OpenWHE.BE  CloseWHE.BE  HighWHRL3.SA  LowWHRL3.SA  OpenWHRL3.SA  \
0       0.436        0.436      2.339524     2.171429      2.171429   
1       0.436        0.436      2.342222     2.125000      2.176667   
2       0.436        0.436      2.381364     2.211818      2.256364   
3       0.436        0.436      2.509000     2.432500      2.461000   
4       0.436        0.436      4.956818     4.403182      4.686818   

   CloseWHRL3.SA    High^IXIC     Low^IXIC    Open^IXIC   Close^IXIC  
0       2.188571  2465.036011  2436.571997  2451.197498  2453.187000  
1       2.176111  2491.339458  2466.998420  2482.686845  2479.864194  
2       2.300909  2414.307273  2386.536366  2401.696811  2401.490867  
3       2.471000  2506.404468  2486.724036  2496.387500  2499.568005  
4       4.654091  2572.418191  2548.061357  2559.358165  2562.140902  

[5 rows x 3046 columns]

In [11]:
final_df.to_csv('Finance_Final_data_sanity_check.csv',index=False)

In [12]:
print('Splitting data into Train, Validation and Test...')
train, validate, test = train_valid_test_split (final_df,train_size,val_size)
print(f'Length of train: {len(train)} \nLength of validation: {len(validate)}\nLength of test: {len(test)}')

Splitting data into Train, Validation and Test...
Length of train: 120 
Length of validation: 8
Length of test: 6


In [13]:
train.to_csv('train.csv',index=False)
validate.to_csv('validate.csv',index=False)
test.to_csv('test.csv',index=False)